In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_csv('GSAF5.csv', encoding = 'iso-8859-1')

In [3]:
data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [4]:
data.shape

(5992, 24)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5992 entries, 0 to 5991
Data columns (total 24 columns):
Case Number               5992 non-null object
Date                      5992 non-null object
Year                      5992 non-null int64
Type                      5992 non-null object
Country                   5949 non-null object
Area                      5590 non-null object
Location                  5496 non-null object
Activity                  5465 non-null object
Name                      5792 non-null object
Sex                       5425 non-null object
Age                       3311 non-null object
Injury                    5965 non-null object
Fatal (Y/N)               5973 non-null object
Time                      2779 non-null object
Species                   3058 non-null object
Investigator or Source    5977 non-null object
pdf                       5992 non-null object
href formula              5991 non-null object
href                      5989 non-null object
C

In [6]:
#eliminamos casos duplicados, contando casos antes y despues para ver cuantos eliminamos
before = len(data)
data = data.drop_duplicates()
after = len(data)
print('Number of duplicate records dropped: ', str(before - after))
#una vez reduzcamos columnas, lo haremos otra vez

Number of duplicate records dropped:  0


In [7]:
#creo un nuevo dataframe con las columnas que considero que seran mas utiles

data_info = data[['Case Number','Year','Type','Country','Activity','Fatal (Y/N)']]
data_info.head()

,Case Number,Year,Type,Country,Activity,Fatal (Y/N)
0,2016.09.18.c,2016,Unprovoked,USA,Surfing,N
1,2016.09.18.b,2016,Unprovoked,USA,Surfing,N
2,2016.09.18.a,2016,Unprovoked,USA,Surfing,N
3,2016.09.17,2016,Unprovoked,AUSTRALIA,Surfing,N
4,2016.09.15,2016,Unprovoked,AUSTRALIA,Surfing,N


In [8]:
#Cambio nombres de columnas a otros mas manejables

data_info = data_info.rename(columns={'Case Number':'Case_num', 'Fatal (Y/N)':'Fatal'})
data_info.columns

Index(['Case_num', 'Year', 'Type', 'Country', 'Activity', 'Fatal'], dtype='object')

In [9]:
data_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5992 entries, 0 to 5991
Data columns (total 6 columns):
Case_num    5992 non-null object
Year        5992 non-null int64
Type        5992 non-null object
Country     5949 non-null object
Activity    5465 non-null object
Fatal       5973 non-null object
dtypes: int64(1), object(5)
memory usage: 327.7+ KB


In [10]:
#reordeno las columnas
column_order = ['Case_num', 'Year', 'Country', 'Activity', 'Type','Fatal']
data_info = data_info[column_order]
data_info.head()

,Case_num,Year,Country,Activity,Type,Fatal
0,2016.09.18.c,2016,USA,Surfing,Unprovoked,N
1,2016.09.18.b,2016,USA,Surfing,Unprovoked,N
2,2016.09.18.a,2016,USA,Surfing,Unprovoked,N
3,2016.09.17,2016,AUSTRALIA,Surfing,Unprovoked,N
4,2016.09.15,2016,AUSTRALIA,Surfing,Unprovoked,N


In [11]:
#repetimos el paso de eliminar filas duplicadas

before = len(data_info)

data_info = data_info.drop_duplicates()

after = len(data_info)

print('Number of duplicate records dropped: ', str(before - after))

#al reducir las columnas, ahora si hay dos filas duplicadas que se eliminan

Number of duplicate records dropped:  2


In [12]:
#busco valores nulos por columna

null_cols = data_info.isnull().sum()
null_cols[null_cols > 0]

#el que mas tiene no llega al 10%, no elimino ninguna columna

Country      43
Activity    526
Fatal        19
dtype: int64

In [14]:

#La gran mayoria de las observaciones se concentran en los ultimos 100 años

all_years = len(data_info['Year'])
print('Todos los registros: ',all_years)

from_1900 = len(data_info.loc[data_info['Year']>1900])
print('Registros desde 1900: ',from_1900)




#Teniendo en cuenta que los registros mas antiguos son los que tienen datos menos fiables y donde faltan mas datos,
#me quedo con solamente los mas recientes

data_info = data_info.loc[data_info['Year']>1900]

Todos los registros:  5990
Registros desde 1900:  5309


In [15]:
#La columna Fatal solamente tiene 19 valores nulos.
#Busco descripcion de las lesiones en la tabla data original para ver si puedo deducir si es fatal o no

data.loc[data['Fatal (Y/N)'].isnull(),['Injury']]

#con estos datos seria factible rellenar los vacios

,Injury
54,"No injury, but sharks repeatedly hit their fin..."
1844,Reported as shark attack but probable drowning
2449,FATAL
3280,"Diver shot the shark, then it injured his arm ..."
3435,"Disappeared, probable drowning but sharks in a..."
3901,Boat damaged
4107,No injury to occupants. Shark tore nets & traw...
4112,Human remains found in shark
5307,"Disappeared, but shark involvement unconfirmed"
5437,"No injury, no attack"


In [16]:
#compruebo los valores en la columna Fatal
#los valores Y y N seria mejor que fueran 1 y 0

data_info['Fatal'].value_counts(dropna=False)

N          4019
Y          1197
UNKNOWN      74
NaN           9
 N            8
F             1
N             1
Name: Fatal, dtype: int64

In [17]:
#Cambio valores que no son Y o N NaN. Tengo que hacerlo en 2 pasos porque el metodo no me admite
#reemplazar por la funcion np.nan. Primero los sustituyo por ''

data_info['Fatal'] = data_info['Fatal'].str.replace('[^YN]', '')
data_info['Fatal'].value_counts(dropna=False)

N      4028
Y      1197
NNN      74
NaN       9
          1
Name: Fatal, dtype: int64

In [18]:
#Los que eran UNKNOWN se han quedado en NNN
#los sustituyo tambien por ''

data_info['Fatal'] = data_info['Fatal'].str.replace('NNN', '')
data_info['Fatal'].value_counts(dropna=False)

N      4028
Y      1197
         75
NaN       9
Name: Fatal, dtype: int64

In [19]:
#ahora si, los valores '' pasan a NaN

data_info['Fatal'] = data_info['Fatal'].replace('', np.nan)
data_info['Fatal'].value_counts(dropna=False)

N      4028
Y      1197
NaN      84
Name: Fatal, dtype: int64

In [20]:
#ahora si cambio Y y N por 1 y 0

data_info.loc[data_info['Fatal'] == 'Y', 'Fatal'] = 1
data_info.loc[data_info['Fatal'] == 'N', 'Fatal'] = 0
data_info['Fatal'].value_counts(dropna=False)

 0.0    4028
 1.0    1197
NaN       84
Name: Fatal, dtype: int64

In [21]:
#Los ataques de tiburones son mayoritariamente letales?

dead = data_info.loc[data_info['Fatal'] == 1]
print('Dead after attack: ',len(dead))

alive = data_info.loc[data_info['Fatal'] == 0]
print('Alive after attack: ',len(alive))

print('Percentage of fatal attacks: ', round(100*len(dead)/(len(dead)+len(alive))),'%')

#Solamente el 23% lo son

Dead after attack:  1197
Alive after attack:  4028
Percentage of fatal attacks:  23 %


In [24]:
#agrupamos por paises y contamos casos
cases_country = data_info.groupby('Country',as_index=False)['Case_num'].count()
cases_country.sort_values(by=['Case_num'], ascending = False).head(10)

#USA y Australia son los paises con mas casos

,Country,Case_num
167,USA,1984
10,AUSTRALIA,1121
140,SOUTH AFRICA,532
119,PAPUA NEW GUINEA,129
18,BRAZIL,100
102,NEW ZEALAND,94
12,BAHAMAS,90
88,MEXICO,73
68,ITALY,61
126,REUNION,55


In [23]:
#Compruebo valores de la columna Type
data_info['Type'].value_counts()

Unprovoked      3873
Provoked         524
Invalid          448
Boat             194
Sea Disaster     181
Boating           89
Name: Type, dtype: int64

In [25]:
#ejemplo de filtrado: ataques no provocados en USA

cases_usa = data_info[(data_info['Country']=='USA')]

print('Cases from USA: ',len(cases_usa))

filtered = data_info.loc[(data_info['Country']=='USA') & 
                (data_info['Type']=='Unprovoked'),]
                

print('Unprovoked cases from USA: ',len(filtered))
print('Percentage of unprovoked cases in the USA: ', round(100*(len(filtered)/len(cases_usa))), '%')

#la mayoria de los casos en USA fueron no provocados

Cases from USA:  1984
Unprovoked cases from USA:  1556
Percentage of unprovoked cases in the USA:  78 %


In [26]:
#Comparamos con el mismo caso en Australia

cases_aus = data_info[(data_info['Country']=='AUSTRALIA')]

print('Cases from Australia: ',len(cases_aus))

filtered2 = data_info.loc[(data_info['Country']=='AUSTRALIA') & 
                (data_info['Type']=='Unprovoked'),]
                

print('Unprovoked cases from Australia: ',len(filtered2))
print('Percentage of unprovoked cases in the Australia: ', round(100*(len(filtered2)/len(cases_aus))), '%')

#Los porcentajes son bastante parecidos

Cases from Australia:  1121
Unprovoked cases from Australia:  779
Percentage of unprovoked cases in the Australia:  69 %


In [53]:
#Compruebo valores de la columna Activity
data_info['Activity'].value_counts()


#El surf y la natacion parecen ser las actividades con mas ataques de tiburon

Surfing                                                                                                                                                                                                                    900
Swimming                                                                                                                                                                                                                   711
Fishing                                                                                                                                                                                                                    366
Spearfishing                                                                                                                                                                                                               321
Wading                                                                                                      

In [56]:
#Miro cual de las 2 actividades tiene mas probabilidad de acabar en muerte tras un ataque

cases_surf = data_info[(data_info['Activity']=='Surfing')]
print('Surfers attacked: ',len(cases_surf))

cases_swim = data_info[(data_info['Activity']=='Swimming')]
print('Swimmers attacked: ',len(cases_swim))

fatal_surf = data_info[(data_info['Activity']=='Surfing')& (data_info['Fatal']==1)]
print('Dead surfers: ',len(fatal_surf))

fatal_swim = data_info[(data_info['Activity']=='Swimming')& (data_info['Fatal']==1)]
print('Dead swimmers: ',len(fatal_swim))

print('Percentage of fatal attacks to surfers: ', round(100*(len(fatal_surf)/len(cases_surf))), '%')
print('Percentage of fatal attacks to swimmers: ', round(100*(len(fatal_swim)/len(cases_swim))), '%')

#Conclusion: un ataque a un nadador es mortal con mas frecuencia que un ataque a un surfista


Surfers attacked:  900
Swimmers attacked:  711
Dead surfers:  49
Dead swimmers:  260
Percentage of fatal attacks to surfers:  5 %
Percentage of fatal attacks to swimmers:  37 %


In [59]:
data_info.to_csv('tidy_shark_dataset.csv')